# 3. 使用 SageMaker 进行超参优化

要第二步的实验中，我们应当成功训练了一个机器学习模型进行交易，但交易的结果可能并不理想。因此在这个 NoteBook 中，我们将使用 SageMaker 的超参优化能力模型进行调优，以试图改善交易的结果。

要完成这个实验，请选择 conda_python3 环境。

在实验开始之前，首先应当先确保环境中安装有火币的 API：

In [ ]:
!git clone https://github.com/HuobiRDCenter/huobi_Python
directory = '/home/ec2-user/SageMaker/sagemaker-huobi-workshop'  # '/root/sagemaker-huobi-workshop' for SageMaker Studio
!cd {directory}/huobi_Python && python3 setup.py -q install
!pip show huobi-client
import os

os._exit(00)

## 准备工作

在这个部分我们将配置变量并准备环境。

首先，应当确保环境变量中有 SageMaker 实例的默认路径：

In [ ]:
import sys

directory = '/home/ec2-user/SageMaker/sagemaker-huobi-workshop'  # '/root/sagemaker-huobi-workshop' for SageMaker Studio
if directory not in sys.path:
    print(directory, 'added to sys.path')
    sys.path.append(directory)
    
prefix = 'hyperparameter_optimization'

接下来，配置一些 SageMaker 训练任务相关的变量：

In [ ]:
import boto3
import datetime
import os
import pandas as pd
import pytz
import sagemaker
from sagemaker import get_execution_role

role = get_execution_role()
session = sagemaker.Session()
aws_default_region = session.boto_session.region_name
aws_account_id = session.boto_session.client('sts').get_caller_identity()['Account']

s3 = boto3.client('s3')
bucket = session.default_bucket()
print('存储桶：', bucket)

model_name = 'prediction'
print('模型：', model_name)

output_location = 's3://{}/{}/output'.format(bucket, model_name)
print('输出路径：', output_location)

image_name = 'huobi'
image_tag = 'latest'
image_uri = '{}.dkr.ecr.{}.amazonaws.com/{}:{}'.format(aws_account_id, aws_default_region, model_name, image_tag)
print('镜像：', image_uri)

接下，获取火币的原始行情数据，并存储到 S3 桶的指定目录中：

In [ ]:
from huobi.client.market import MarketClient
from huobi.constant import *
from huobi.utils import *

# get data from huobi
market_client = MarketClient(init_log=True)
interval = CandlestickInterval.DAY1
symbol = "btcusdt"

flag = True
while flag:
    try:
        list_obj = market_client.get_candlestick(symbol, interval, 1300)
        # LogInfo.output("---- {interval} candlestick for {symbol} ----".format(interval=interval, symbol=symbol))
        # LogInfo.output_list(list_obj)
        flag = False
        print('Data load success')
    except:
        continue

# transform huobi data to Pandas DataFrame
columns = ['tradedate', 'high', 'low', 'open', 'close', 'count', 'amount', 'volume']
df = pd.DataFrame([[i.id, i.high, i.low, i.open, i.close, i.count, i.amount, i.vol] for i in list_obj], columns=columns)
timezone = pytz.timezone('Asia/Shanghai')
df['tradedate'] = df['tradedate'].apply(lambda x: datetime.datetime.fromtimestamp(x).astimezone(timezone).strftime('%Y-%m-%d'))
df.set_index('tradedate', inplace=True)
df.sort_index(inplace=True)

start_date = df.index[0]
end_date = df.index[-1]
print('Sample range:', start_date, '-', end_date)

# save data to s3
s3.put_object(Body=df.to_csv(), Bucket=bucket, Key='{}/input/data_raw.csv'.format(model_name))
raw_data_path = 's3://{}/{}/input/data_raw.csv'.format(bucket, model_name)

## 测试自定义算法容器

在这个演示中，我们将使用 SageMaker 的超参优化功能。这需要将算法打包至容器中在 SageMaker 上运行。在此之前，可以先在本地构建镜像，并且使用默认参数简单测试自定义算法容器是否可以正常运行。

本实验中提供了一个示例 TensorFlow 自定义 Dockerfile。可以运行以下命令在本地环境进行 build：

In [ ]:
# !sudo docker stop $(sudo docker ps -a -q)
# !sudo docker rm $(sudo docker ps -a -q)
# !sudo docker rmi $(sudo docker images)
# !docker ps -a
# !docker images
!cd {directory} && sudo docker build {prefix} -t {image_name}:{image_tag}

镜像创建成功后，我们可以在本地环境运行镜像，测试是否能够成功运行：

In [ ]:
# default parameters
params = { 
    "long_threshold" : 0.5,
    "short_threshold" : 1,
    "profit_target" : 0.02,
    "stop_target" : 0.01,
    "repeat_count": 20,
    "repeat_step": 1,
    "forward_window": 10
}

base_job_name = model_name

estimator = sagemaker.estimator.Estimator(
    'huobi:latest',
    role=role,
    train_instance_count=1,
    train_instance_type='local',
    output_path=output_location,
    base_job_name=base_job_name, 
    hyperparameters=params,
)

estimator.fit(raw_data_path)

在确认容器能够成功运行后，我们需要将容器推送至 ECR 镜像仓库，以便在超参优化任务中使用：

In [ ]:
# create ECR repository
!aws ecr create-repository --repository-name {model_name} --region {aws_default_region} > /dev/null

In [ ]:
exist_image = !docker images -q {image_name}:{image_tag} 2> /dev/null
if len(exist_image) > 0:
    !docker tag {image_name}:{image_tag} {image_uri}
!$(aws ecr get-login --region {aws_default_region} --no-include-email)
print('Pushing image')
!docker push {image_uri}
print('Done')

## 超参优化

在开始之前，我们需要花一些时间来进行超参优化任务的配置。这些配置主要包括超参的定义和取值范围、目标参数的定义以及从日志中提取目标参数所需的正则表达式。这些配置信息通过 JSON 格式定义：

In [ ]:
# 超参优化任务名称
tuning_job_name = model_name

from sagemaker.tuner import IntegerParameter, CategoricalParameter, ContinuousParameter, HyperparameterTuner

# 超参调优的参数配置
hyperparameter_ranges = {
    'long_threshold': ContinuousParameter(0.1, 0.9),
    'short_threshold': ContinuousParameter(1.0, 1.1),
    'profit_target': ContinuousParameter(0.01, 0.1),
    'stop_target': ContinuousParameter(0.01, 0.1),
    'repeat_count': IntegerParameter(10, 30),
    'repeat_step': IntegerParameter(1, 2),
    'forward_window': IntegerParameter(5, 30)
}

# 目标参数的配置
metric_definitions = [
    {
        'Name': 'loss', 
        'Regex': 'loss: ([0-9\\.]+)%'
    },
    {
        'Name': 'accuracy', 
        'Regex': 'accuracy: ([0-9\\.]+)%'
    },
    {
        'Name': 'train:accuracy', 
        'Regex': 'Training Data accuracy: ([0-9\\.]+)%'
    },
    {
        'Name': 'test:accuracy', 
        'Regex': 'Test Data accuracy: ([0-9\\.]+)%'
    }
]
objective_metric_name = "test:accuracy"

接下来我们将在 SageMaker 中启动一个超参优化任务来寻找最优的参数组合。每一个超参优化任务会并行启动多个容器进行模型训练和测试，任务可能会持续至少数十分钟。我们可以尝试挑战超参优化中的一些参数来提升运行的效率，例如参数的取值范围、任务节点数量、训练任务总数等。

In [ ]:
estimator = sagemaker.estimator.Estimator(
    image_uri,
    role,
    train_instance_count=1,
    train_instance_type='ml.m5.4xlarge',
    output_path=output_location,
    sagemaker_session=session
)

tuner = sagemaker.tuner.HyperparameterTuner(
    estimator, 
    objective_metric_name,
    hyperparameter_ranges,
    metric_definitions=metric_definitions,
    strategy='Bayesian',
    objective_type='Maximize', 
    max_jobs=100,
    max_parallel_jobs=10,
    base_tuning_job_name=tuning_job_name,
    early_stopping_type='Auto'
)

tuner.fit(raw_data_path)

在超参优化任务结束之后，我们可以通过以下代码查看优化的结果,并选取一个理想的模型：

In [ ]:
tuner_analytics = tuner.analytics()
tuning_result = tuner_analytics.dataframe().sort_values(['FinalObjectiveValue'], ascending=False)

# select desired training result
i = 0
selected_training_job = tuning_result["TrainingJobName"].iloc[i]

params = { 
    "long_threshold" : float(tuning_result["long_threshold"].iloc[i]),
    "short_threshold" : float(tuning_result["short_threshold"].iloc[i]),
    "profit_target" : float(tuning_result["profit_target"].iloc[i]),
    "stop_target" : float(tuning_result["stop_target"].iloc[i]),
    "repeat_count": int(tuning_result["repeat_count"].iloc[i]),
    "repeat_step": int(tuning_result["repeat_step"].iloc[i]),
    "forward_window": int(tuning_result["forward_window"].iloc[i]),
}

import json
hyperparameter_path = '{}/hyperparameters.json'.format(directory)
print('超参路径：', hyperparameter_path)
with open(hyperparameter_path, 'w') as fp:
    json.dump(params, fp)

tuning_result.head(10)

训练完成后的模型由 SageMaker 自动打包并上传至 S3。我们只需找到最优的训练任务，并将其生产的 "model.h5" 的模型文件解压缩到实验目录下：

In [ ]:
import boto3
import io
import tarfile

s3_output_path = '{}/output/{}/output/model.tar.gz'.format(model_name, selected_training_job)
print('s3路径：', s3_output_path)

bytestream = io.BytesIO(s3.get_object(Bucket=bucket, Key=s3_output_path)['Body'].read())
compressed_file = tarfile.open(fileobj=bytestream)
extract_path = '{}/'.format(directory)
compressed_file.extractall(path=extract_path)
model_path = extract_path + 'model.h5'
print('模型路径：', model_path)

模型和超参保存成功后，就可以第二个 Notebook 的“回测”步骤对模型再进行验证了。